# `nanoGPT`

## Install / Setup

### First Time Running

We need to install `ngpt` and setup the Shakespeare dataset

This will need to be ran the first time you are running this notebook.

Following the

```python
!python3 -m pip install nanoGPT
```

you will need to restart your runtime (Runtime -> Restart runtime)

After this, you should be able to

```python
>>> import ngpt
>>> ngpt.__file__
'/content/nanoGPT/src/ngpt/__init__.py'
```

In [1]:
%%bash

[ $(python3 -c 'import ngpt; print(ngpt.__file__)') ] && export HAS_NGPT=1 || export HAS_NGPT=0
[ $HAS_NGPT ] || git clone https://github.com/saforem2/nanoGPT;
[ $HAS_NGPT ] || python3 nanoGPT/data/shakespeare_char/prepare.py
[ $HAS_NGPT ] || python3 -m pip install -e nanoGPT -vvv

## Post Install

If installed correctly, you should be able to:

```python
>>> import ngpt
>>> ngpt.__file__
'/path/to/nanoGPT/src/ngpt/__init__.py'
```

In [2]:
%load_ext autoreload
%autoreload 2

import ngpt
from rich import print
print(ngpt.__file__)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


/lus/grand/projects/datascience/foremans/locations/thetaGPU/projects/saforem2/nanoGPT/src/ngpt/__init__.py

## Build Trainer

Explicitly, we:

1. `setup_torch(...)`
2. Build `cfg: DictConfig = get_config(...)`
3. Instnatiate `config: ExperimentConfig = instantiate(cfg)`
4. Build `trainer = Trainer(config)`

In [3]:
import os
from ezpz import setup_torch
from hydra.utils import instantiate
from ngpt.configs import get_config, PROJECT_ROOT
from ngpt.trainer import Trainer

HF_DATASETS_CACHE = PROJECT_ROOT.joinpath('.cache', 'huggingface')
HF_DATASETS_CACHE.mkdir(exist_ok=True, parents=True)

os.environ['MASTER_PORT'] = '5631'
os.environ['HF_DATASETS_CACHE'] = HF_DATASETS_CACHE.as_posix()

rank = setup_torch('DDP', seed=1234)
cfg = get_config(
    [
        'data=owt',
        'model=gpt2_large',
        'optimizer=gpt2_large',
        'train=gpt2_large',
        'train.init_from=gpt2-large',
        'train.max_iters=1000',
        'train.dtype=bfloat16',
    ]
)
config = instantiate(cfg)
trainer = Trainer(config)

--------------------------------------------------------------------------

  Local host:   thetagpu24
  Local device: mlx5_0
--------------------------------------------------------------------------
RANK: 0 / 0


[2023-11-10 15:54:40][WARNING][configs.py:298] - No meta.pkl found, assuming GPT-2 encodings...
[2023-11-10 15:54:40][INFO][configs.py:264] - Rescaling GAS -> GAS // WORLD_SIZE = 1 // 1
[2023-11-10 15:54:40][INFO][configs.py:399] - Tokens per iteration: 4,096
[2023-11-10 15:54:40][INFO][configs.py:431] - Using <torch.amp.autocast_mode.autocast object at 0x7f1ff0237820>
[2023-11-10 15:54:40][INFO][trainer.py:184] - Initializing from OpenAI GPT-2 Weights: gpt2-large


2023-11-10 15:54:40.683311: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


[2023-11-10 15:54:44,413] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2023-11-10 15:54:45][INFO][model.py:225] - loading weights from pretrained gpt: gpt2-large
[2023-11-10 15:54:45][INFO][model.py:234] - forcing vocab_size=50257, block_size=1024, bias=True
[2023-11-10 15:54:45][INFO][model.py:240] - overriding dropout rate to 0.0
[2023-11-10 15:54:56][INFO][model.py:160] - number of parameters: 772.72M


[2023-11-10 15:55:18][INFO][model.py:290] - num decayed parameter tensors: 146, with 773,428,480 parameters
[2023-11-10 15:55:18][INFO][model.py:291] - num non-decayed parameter tensors: 290, with 601,600 parameters
[2023-11-10 15:55:19][INFO][model.py:297] - using fused AdamW: True


Process ForkProcess-15:
Process ForkProcess-6:
Process ForkProcess-28:
Process ForkProcess-24:
Process ForkProcess-9:
Process ForkProcess-18:
Process ForkProcess-20:
Process ForkProcess-11:
Process ForkProcess-26:
Process ForkProcess-23:
Process ForkProcess-31:
Process ForkProcess-12:
Process ForkProcess-27:
Process ForkProcess-22:
Process ForkProcess-32:
Process ForkProcess-16:
Process ForkProcess-8:
Process ForkProcess-3:
Process ForkProcess-30:
Process ForkProcess-25:
Process ForkProcess-7:
Process ForkProcess-21:
Process ForkProcess-29:
Process ForkProcess-19:
Process ForkProcess-1:
Process ForkProcess-13:
Process ForkProcess-10:
Process ForkProcess-17:
Process ForkProcess-5:
Process ForkProcess-14:
Process ForkProcess-2:
Process ForkProcess-4:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call las

## Train Model


|  **NAME**  |     **DESCRIPTION**          |
|:----------:|:----------------------------:|
|   `step`   | Current training step        |
|   `loss`   | Loss value                   |
|   `dt`     | Time per step (in **ms**)    |
|   `sps`    | Samples per second           |
|   `mtps`   | (million) Tokens per sec     |
|   `mfu`    | Model Flops Utilization*     |
^Logging Legend

*in units of A100 `bfloat16` peak FLOPS

In [4]:
trainer.train()

  0%|          | 0/1000 [00:00<?, ?it/s]

[2023-11-10 15:57:25][INFO][trainer.py:540] - step=100 loss=2.618 dt=571.254 sps=1.751 mtps=0.007 mfu=35.869 train_loss=2.675 val_loss=2.652
[2023-11-10 15:58:20][INFO][trainer.py:540] - step=200 loss=2.514 dt=584.762 sps=1.710 mtps=0.007 mfu=35.786 train_loss=2.675 val_loss=2.652
[2023-11-10 15:59:15][INFO][trainer.py:540] - step=300 loss=2.594 dt=539.828 sps=1.852 mtps=0.008 mfu=36.003 train_loss=2.675 val_loss=2.652
[2023-11-10 16:00:10][INFO][trainer.py:540] - step=400 loss=2.673 dt=549.861 sps=1.819 mtps=0.007 mfu=36.129 train_loss=2.675 val_loss=2.652
[2023-11-10 16:01:04][INFO][trainer.py:540] - step=500 loss=2.701 dt=534.227 sps=1.872 mtps=0.008 mfu=36.351 train_loss=2.675 val_loss=2.652
[2023-11-10 16:01:59][INFO][trainer.py:540] - step=600 loss=2.769 dt=567.887 sps=1.761 mtps=0.007 mfu=36.324 train_loss=2.675 val_loss=2.652
[2023-11-10 16:02:53][INFO][trainer.py:540] - step=700 loss=2.766 dt=544.301 sps=1.837 mtps=0.008 mfu=36.456 train_loss=2.675 val_loss=2.652
[2023-11-10 1

## Evaluate Model

In [5]:
query = "What is a supercomputer?"
outputs = trainer.evaluate(query, num_samples=1, display=False)

In [6]:
from rich.text import Text
from enrich.console import get_console
console = get_console()

console.print(fr'\[prompt]: "{query}"')
console.print("\[response]:\n\n" + fr"{outputs['0']['raw']}")

[prompt]: "What is a supercomputer?"

[response]:

What is a supercomputer? A supercomputer is a computer, but one with a faster processor than the ones we have today.

But why do we need them? Why do they make sense?

Here’s what a supercomputer looks like, from the perspective of a computer scientist.

As you can see, he’s a computer scientist, and that’s why he was able to solve the problem in a breakthrough way.

But today, unlike the supercomputers of the past, the computers of the future will be able to do so much more.

We’ll explain why.

Do you want to learn more about quantum physics? Join us on the quantum physics / computer science course to get an overview of the quantum physics and the big challenges in the lab.

Science and technology are just like magic.

And that’s why we’re so excited about VR in general.

VR in general makes a quantum leap.

Like magic

Erik Dorsgaard, a quantum physicist and NASA Ames Research Center engineer, heads up a team of researchers in the field that’s working on the implementation of the Oculus Rift and HTC Vive VR experiences.

“VR is exciting, but VR is the one that takes us to quantum physics,” Dorsgaard says. “VR is exciting, but VR is the one that takes us to quantum physics.”

Dorsgaard says that quantum physics is the science of order, and quantum physics is what makes a quantum system tick: it is the physics of chaos. “In quantum physics, chaos is chaos,” Dorsgaard says. “So quantum physics researchers are interested in chaos.”

To understand how quantum physics works, you need to know some basics. First, quantum physics is a physics of particles. Quantum particles are particles with different spin and spin directions.

“This has a very simple idea,” says Dorsgaard. “Think of a particle as two particles; one is a spin particle and one is a spin particle. The particle with spin is called a spin particle. And when we take a quantum particle with spin, we have to ask, ‘Is this particle spin or spin?’”

Dorsgaard suggests that when we take a spin particle and put it into a quantum system, we can

Exception ignored in atexit callback: <bound method InteractiveShell.atexit_operations of <ipykernel.zmqshell.ZMQInteractiveShell object at 0x7f21b0b08940>>

Traceback (most recent call last):

File "/lus/grand/projects/datascience/foremans/locations/thetaGPU/projects/saforem2/nanoGPT/venvs/thetaGPU/2023-01-11/lib/python3.10/site-packages/rich/jupyter.py", line 91, in display

...

File "/lus/grand/projects/datascience/foremans/locations/thetaGPU/projects/saforem2/nanoGPT/venvs/thetaGPU/2023-01-11/lib/python3.10/site-packages/rich/console.py", line 860, in __enter__

self._enter_buffer()

File "/lus/grand/projects/datascience/foremans/locations/thetaGPU/projects/saforem2/nanoGPT/venvs/thetaGPU/2023-01-11/lib/python3.10/site-packages/rich/console.py", line 818, in _enter_buffer

self._buffer_index += 1

RecursionError: maximum recursion depth exceeded